In [321]:
import numpy as np
import pandas as pd

#pd.set_option('display.max_rows', 500)

In [322]:
# Abre os arquivos
treino = pd.read_csv('train.csv')
teste = pd.read_csv('test.csv')

# Damos uma espiada no data frame
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Sempre legal dar uma olhada nas estatísticas do data frame. Aqui já temos noção de como os dados estão, por exemplo se tem Age negativos, ou a Age com mais de 200 anos.. ja pensou?

In [323]:
treino.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Depois que a gente conheceu os dados de treino, vamos ver sem tem features com valures NULL

In [324]:
treino.isnull().sum().sort_values()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
Age            177
Cabin          687
dtype: int64

In [325]:
teste.isnull().sum().sort_values()

PassengerId      0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Embarked         0
Fare             1
Age             86
Cabin          327
dtype: int64

Idade nula é um porblema. Vamos resolver isso assim: Subsituir os nulos pela média, mas vamos fazer de forma aleatória levando em conta o range dentro do desvio padrão.

In [326]:
# TREINO
media = treino['Age'].mean()
desvio = treino['Age'].std()
total_nulos = treino['Age'].isnull().sum()

# Cria um array com a quantidade de Ages em branco
total_nulos_randomicos = np.random.random_integers(media-desvio, media+desvio,size=total_nulos)
# Aplica o array nos registros do data frame que estao com idade null
treino['Age'][np.isnan(treino['Age'])] = total_nulos_randomicos

# APLICA NO TESTE
media = teste['Age'].mean()
desvio = teste['Age'].std()
total_nulos = teste['Age'].isnull().sum()

# Cria um array com a quantidade de Ages em branco
total_nulos_randomicos = np.random.random_integers(media-desvio, media+desvio,size=total_nulos)
# Aplica o array nos registros do data frame que estao com idade null
teste['Age'][np.isnan(teste['Age'])] = total_nulos_randomicos


In [327]:
# Vamos ver se deu bom...
print('Age null em treino: ', treino['Age'].isnull().sum())
print('Age null em teste: ', teste['Age'].isnull().sum())

Age null em treino:  0
Age null em teste:  0


O campo sexo precisa ser transformado em binário.

In [328]:
# Tratamentos
treino['Sex_binario'] = treino['Sex'].apply(lambda x: 1 if x == 'female' else 0)
teste['Sex_binario'] = teste['Sex'].apply(lambda x: 1 if x == 'female' else 0)

Vamos jogar os caras null para -1

In [329]:
treino.fillna(value=-1)
teste.fillna(value=-1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_binario
0,892,3,"Kelly, Mr. James",male,34.50,0,0,330911,7.8292,-1,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00,1,0,363272,7.0000,-1,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.00,0,0,240276,9.6875,-1,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.00,0,0,315154,8.6625,-1,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00,1,1,3101298,12.2875,-1,S,1
5,897,3,"Svensson, Mr. Johan Cervin",male,14.00,0,0,7538,9.2250,-1,S,0
6,898,3,"Connolly, Miss. Kate",female,30.00,0,0,330972,7.6292,-1,Q,1
7,899,2,"Caldwell, Mr. Albert Francis",male,26.00,1,1,248738,29.0000,-1,S,0
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.00,0,0,2657,7.2292,-1,C,1
9,901,3,"Davies, Mr. John Samuel",male,21.00,2,0,A/4 48871,24.1500,-1,S,0


Vamos criar umas features... pensar em algumas hipóteses.

TREINO

In [330]:
# Se reparar nos nomes, existem alguns títulos.
treino['is_mr'] = treino['Name'].str.contains('Mr.')
treino['is_mrs'] = treino['Name'].str.contains('Mrs.')
treino['is_miss'] = treino['Name'].str.contains('Miss.')
treino['is_rev'] = treino['Name'].str.contains('Rev.')
treino['is_master'] = treino['Name'].str.contains('Master.')
treino['is_dr'] = treino['Name'].str.contains('Dr.')

# Junta a informação se o passageiro esta sozinho, com filhos e/ou marido. Seria o total de pessoas da familia viajando juntos, algo assim.
treino['relatives'] = treino.apply(lambda row: int(row['SibSp'] + row['Parch'])>0, axis=1)

# Uma classificação por faixa etária
treino['faixa_idade'] = treino['Age']/12
# Se é criança
treino['is_children'] = treino.apply(lambda row: row['Age']<15, axis=1)
# Se é idoso
treino['is_old'] = treino.apply(lambda row: row['Age']>60, axis=1)

# Prmeira letra da cabine, imaginei que isso pode influenciar pelo fato de estar no inicio/final de corredor, andar diferente, enfim... 
treino['cabin_fst_letter'] = treino['Cabin'].str[0:1]
treino['cabin_fst_letter'].replace(np.nan, -1, inplace=True) # soh acerta o que ficou como NULL

treino['cabine_is_null'] = treino['Cabin'].isnull().astype(int)

treino['embarked_s'] = (treino['Embarked'] == 'S').astype(int)
treino['embarked_c'] = (treino['Embarked'] == 'C').astype(int)

Vamos criar umas features... pensar em algumas hipóteses.

TESTE

In [331]:
# Se reparar nos nomes, existem alguns títulos.
teste['is_mr'] = teste['Name'].str.contains('Mr.')
teste['is_mrs'] = teste['Name'].str.contains('Mrs.')
teste['is_miss'] = teste['Name'].str.contains('Miss.')
teste['is_rev'] = teste['Name'].str.contains('Rev.')
teste['is_master'] = teste['Name'].str.contains('Master.')
teste['is_dr'] = teste['Name'].str.contains('Dr.')

# Junta a informação se o passageiro esta sozinho, com filhos e/ou marido. Seria o total de pessoas da familia viajando juntos, algo assim.
teste['relatives'] = teste.apply(lambda row: int(row['SibSp'] + row['Parch'])>0, axis=1)

# Uma classificação por faixa etária
teste['faixa_idade'] = teste['Age']/12
# Se é criança
teste['is_children'] = teste.apply(lambda row: row['Age']<15, axis=1)
# Se é idoso
teste['is_old'] = teste.apply(lambda row: row['Age']>60, axis=1)

# Prmeira letra da cabine, imaginei que isso pode influenciar pelo fato de estar no inicio/final de corredor, andar diferente, enfim... 
teste['cabin_fst_letter'] = teste['Cabin'].str[0:1]
teste['cabin_fst_letter'].replace(np.nan, -1, inplace=True) # soh acerta o que ficou como NULL

teste['cabine_is_null'] = teste['Cabin'].isnull().astype(int)

teste['embarked_s'] = (teste['Embarked'] == 'S').astype(int)
teste['embarked_c'] = (teste['Embarked'] == 'C').astype(int)

Muito importante dar um check na correlação das variáveis com o nosso target. Vamos usar isso para limpar o data frame ou para excluir alguma feature na hora da predição.

In [332]:
#treino.corr().abs()[["Survived"]]

In [333]:
# Pinto os nomes das colunas pra conseguir copiar e colar abaixo :)
treino.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Sex_binario',
       'is_mr', 'is_mrs', 'is_miss', 'is_rev', 'is_master', 'is_dr',
       'relatives', 'faixa_idade', 'is_children', 'is_old',
       'cabin_fst_letter', 'cabine_is_null', 'embarked_s', 'embarked_c'],
      dtype=object)

In [334]:
# Vou usar essas colunas como preditoras
colunas = ['Sex_binario', 'Pclass',  'Embarked', 'Fare', 'SibSp', 'Parch',
           'is_mr', 'is_mrs', 'is_miss', 'is_rev', 'is_master',
            'is_dr', 'relatives', 'cabin_fst_letter', 'faixa_idade',
            'is_children', 'is_old', 'cabine_is_null', 'embarked_s', 'embarked_c'
            
]

In [335]:
# Transformo algumas features categoricas
for coluna in ['Cabin', 'Embarked', 'cabin_fst_letter']:
    treino[coluna]=pd.Categorical(treino[coluna]).codes
    teste[coluna]=pd.Categorical(teste[coluna]).codes

Agora sim, vamos pa predição

In [336]:
entradas = colunas
saida = 'Survived'

X = treino[entradas]
y = treino[saida]
X = X.fillna(-1)

from sklearn.model_selection import train_test_split


In [337]:
# validacao cruzada
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
print('Colunas:', X.columns.values)
resultados = []
kf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=10)
#modelo = RandomForestClassifier(n_estimators=1000, n_jobs=-1,random_state=1, max_depth=50)
modelo = LogisticRegression()
for linhas_treino, linhas_teste in kf.split(X):

    X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_teste]
    y_treino, y_valid = y.iloc[linhas_treino], y.iloc[linhas_teste]
    
    modelo.fit(X_treino, y_treino)

    p = modelo.predict(X_valid)

    acc = np.mean(y_valid == p)
    print('Acuracia: ', acc)
    resultados.append(acc)
        
print('Media acuracia:',  np.mean(resultados))


Colunas: ['Sex_binario' 'Pclass' 'Embarked' 'Fare' 'SibSp' 'Parch' 'is_mr' 'is_mrs'
 'is_miss' 'is_rev' 'is_master' 'is_dr' 'relatives' 'cabin_fst_letter'
 'faixa_idade' 'is_children' 'is_old' 'cabine_is_null' 'embarked_s'
 'embarked_c']
Acuracia:  0.8251121076233184
Acuracia:  0.797752808988764
Acuracia:  0.8295964125560538
Acuracia:  0.8179775280898877
Acuracia:  0.8183856502242153
Acuracia:  0.802247191011236
Acuracia: 0.8228699551569507
Acuracia:  0.8112359550561797
Acuracia:  0.8340807174887892
Acuracia:  0.8179775280898877
Acuracia:  0.820627802690583
Acuracia:  0.8224719101123595
Acuracia:  0.8430493273542601
Acuracia:  0.8202247191011236
Acuracia: 0.820627802690583
Acuracia:  0.8247191011235955
Acuracia:  0.8408071748878924
Acuracia:  0.8089887640449438
Acuracia:  0.8251121076233184
Acuracia:  0.8359550561797753
Media acuracia: 0.8219909810046857


In [338]:
predict = modelo.predict(teste[entradas].fillna(-1))

# para facilitar o nome do arquivo
file_saida = input('Digite um nome de arquivo para gerar uma submissao: ')

if file_saida != '':
    sub = pd.Series(predict, index=teste['PassengerId'], name='Survived')
    sub.to_csv(file_saida + '.csv')
    print('Arquivo gerado com sucesso :)')

Arquivo gerado com sucesso :)
